In [36]:
import os.path
import io
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive"]


def download_file(real_file_id, download_path='D:/CYLENE/APIDRIVE'):
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("drive", "v3", credentials=creds)

        # Call the Drive v3 API
        results = (
            service.files()
            .list(pageSize=15, fields="nextPageToken, files(id, name)")
            .execute()
        )
        items = results.get("files", [])

        if not items:
            print("No files found.")
            return
        print("Files:")
        for item in items:
            print(f"{item['name']} ({item['id']})")
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f"An error occurred: {error}")

    try:
        file_id = real_file_id
        file_metadata = service.files().get(fileId=file_id).execute()
        
        # Verifica se o arquivo é um documento do Google (por exemplo, Google Docs)
        if 'application/vnd.google-apps.' in file_metadata['mimeType']:
            request = service.files().export_media(
                fileId=file_id, mimeType='application/pdf')
        else:
            request = service.files().get_media(fileId=file_id)
        file_extension = os.path.splitext(file_metadata['name'])[1]
        file_path = os.path.join(download_path, f"{file_id}{file_extension}")

        request = service.files().get_media(fileId=file_id)
        # file_path = os.path.join(download_path, f"{file_id}.pdf")  # Use o formato de arquivo apropriado

        with open(file_path, "wb") as file:
            downloader = MediaIoBaseDownload(file, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print(f"Download {int(status.progress() * 100)}.")

        print(f"O arquivo foi salvo em: {file_path}")
        return file_path

    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f"An error occurred: {error}")

    return None

# https://docs.google.com/document/d/1udEuyBYKB0eWaBHDNpfeXfyuSbj2NCr6uD4MMj2w77A/edit?usp=sharing
# https://drive.google.com/file/d/14s1irRJc6hu9MRR1Qn0go8bnuWOoN6ky/view?usp=drive_link


if __name__ == "__main__":
    download_file(real_file_id="1udEuyBYKB0eWaBHDNpfeXfyuSbj2NCr6uD4MMj2w77A")

Files:
teste py (1udEuyBYKB0eWaBHDNpfeXfyuSbj2NCr6uD4MMj2w77A)
GABARITO 2º DIA 3º SERIE.docx (1QW2Y7ZabXuDo0LE6HvfdK9k7ef3ra2X3)
GABARITO 2º DIA  (1v65BzcDEuayVek3GozJ9y9Acy-uutv-W)
SUMULADO 1º DIA 3º SERIE.docx (1AZ1kJtCPrAU4_O3lzSpQMRODI5_vbkIF)
SIMULADO 2º DIA 3º SERIE.docx (1djSGghcIaX7OQcFKOAJTqdV0F3rBJk7N)
Untitled0.ipynb (1k8-gTYGWGj28Z2IzzJjDjPOlG7a_l7u6)
Gia basico.pdf (14s1irRJc6hu9MRR1Qn0go8bnuWOoN6ky)
Upgrade-correcao-troca-senha.zip (0B6RxwkFtKghvQlc1a2V3RVVnbG8)
RQUALI (1TI1malCuYCIvhsoNO8cWtovYnD0JqC1L)
Equipe/Inovatec/SICAP (12vtnLurfuydrUdP77Hu110AcOAS4DdFk)
plano-de-trabalho-jonas (1Ovvl1JBbgWtw1HiDcYytg3MmfyN8IWdtW_4_XI5QJ7I)
Daniel (1UjLEY-7PyngoQNHz39zg3Qul9Y2zrMiy)
Flávio (1GjOu3SfLOHq41vKQLaZMuCDLWzBZggnG)
Jonas (1gEjxVQC7M1vDPuzA2Z212nPJIcqBqXWu)
Bival (12xL3uNMsCL2ax1NeR8fMfW8j3b7EN6VU)
An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1udEuyBYKB0eWaBHDNpfeXfyuSbj2NCr6uD4MMj2w77A?alt=media returned "Only files with bina